In [1]:
spark

In [2]:
# value info in wikidata entity table (https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Edits/Wikidata_entity)
# is a string as opposed to struct (because it has a variable schema)
# this UDF extracts the QID value (or null if doesn't exist)
def getValue(obj):
    try:
        d =  eval(obj)
        return d.get('id')
    except Exception:
        return None
    
spark.udf.register('getValue', getValue, 'string')

<function __main__.getValue>

In [16]:
snapshot = '2020-11-02'
wiki_db = 'enwiki'

In [20]:
print_for_hive = False
do_execute = True

query = """
WITH relevant_qids AS (
    SELECT DISTINCT item_id
      FROM wmf.wikidata_item_page_link
     WHERE snapshot = '{0}'
           AND page_namespace = 0
           AND wiki_db = '{1}'
),
exploded_statements AS (
    SELECT id as item_id,
           explode(claims) as claim
      FROM wmf.wikidata_entity w
     INNER JOIN relevant_qids q
           ON (w.id = q.item_id)
     WHERE w.snapshot = '{0}'
),
relevant_statements AS (
    SELECT item_id,
           CONCAT_WS(" ", Array(claim.mainSnak.property, getValue(claim.mainSnak.dataValue.value))) AS statement_str
     FROM exploded_statements
)
SELECT item_id,
       CONCAT_WS(" ", COLLECT_LIST(statement_str)) AS statements
  FROM relevant_statements
 GROUP BY item_id
""".format(snapshot, wiki_db)

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    result = spark.sql(query)
    result.write.csv(path="/user/isaacj/wikidata-statements-enwiki", compression="gzip", header=True, sep="\t")


WITH relevant_qids AS (
    SELECT DISTINCT item_id
      FROM wmf.wikidata_item_page_link
     WHERE snapshot = '2020-11-02'
           AND page_namespace = 0
           AND wiki_db = 'enwiki'
),
exploded_statements AS (
    SELECT id as item_id,
           explode(claims) as claim
      FROM wmf.wikidata_entity w
     INNER JOIN relevant_qids q
           ON (w.id = q.item_id)
     WHERE w.snapshot = '2020-11-02'
),
relevant_statements AS (
    SELECT item_id,
           CONCAT_WS(" ", Array(claim.mainSnak.property, getValue(claim.mainSnak.dataValue.value))) AS statement_str
     FROM exploded_statements
)
SELECT item_id,
       CONCAT_WS(" ", COLLECT_LIST(statement_str)) AS statements
  FROM relevant_statements
 GROUP BY item_id

